# Elo score calculator

In [1]:
import pandas as pd
import numpy as np

/var/folders/rm/zd360c_n3394v5zq_gnyj9s00000gn/T/ipykernel_34895/2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Loads the tournaments.

In [2]:
tournaments = pd.read_excel('Data/MK8_Tournaments.xlsx', sheet_name='Scores')
tournaments = tournaments.copy()
tournaments['Date'] = tournaments['Date'].dt.strftime('%Y%m%d').astype(int)
tournaments['Elo'] = 0

tournaments.head()

,Date,Challenger,Round,Pts,Elo
0,20211006,Ilir,1,37,0
1,20211006,Alex,1,27,0
2,20211006,Sylvain,1,29,0
3,20211006,Sacha,1,54,0
4,20211006,Ilir,2,48,0


Creates the players dataframe with initial Elo scores of 1000.

In [3]:
players = pd.DataFrame(tournaments['Challenger'].unique(), columns=['Challenger'])

players['Elo'] = 1000
players['Nb_matches'] = 0
players['Last_update'] = tournaments['Date'].min()

players = players.set_index('Challenger')

players

,Elo,Nb_matches,Last_update
Challenger,,,
Ilir,1000,0,20211006
Alex,1000,0,20211006
Sylvain,1000,0,20211006
Sacha,1000,0,20211006
Julien,1000,0,20211006
Lev,1000,0,20211006
Olivier,1000,0,20211006
Antoine,1000,0,20211006
Rodolphe,1000,0,20211006


In [4]:
def prob(diff):
    return 1/(1 + (10**(-diff/400)))

prob(diff=147)

0.6997694032620582

In [5]:
results = pd.DataFrame(columns=['Date', 'Round', 'Winner', 'Loser', 'Count'])

In [6]:
def compute_elo(df, players) :

    score = df.copy()
    score['Rank'] = score['Pts'].rank(method='dense', ascending=False).astype(int)

    score = pd.merge(score[['Challenger', 'Rank', 'Date']], players['Elo'], on = 'Challenger')
    score.set_index('Challenger', inplace=True)

    # Creating an empty DataFrame for the 2D matrix of probabilities
    probs = pd.DataFrame(index=score.index, columns=score.index)

    # Creating an empty DataFrame for the 2D matrix of results (win/loss/draw)
    wins = pd.DataFrame(index=score.index, columns=score.index)

    game_result = pd.DataFrame(columns=['Winner', 'Loser', 'Count'])

    # Populating the matrices with the 
    for i in score.index:
        for j in score.index:
            # the probability of win is computed using the prob function of the Elo score difference between the players
            probs.at[i, j] = prob(score.at[i, 'Elo'] - score.at[j, 'Elo'])
            # If the player won, he gets a 1, if draw then 0.5 if loss then 0
            wins.at[i, j] = (1 - np.sign(score.at[i, 'Rank'] - score.at[j, 'Rank']))/2
            # Compute the winned matches 
            if i != j :
                game_result.loc[len(game_result)] = [i, j, wins.loc[i,j]]
    
    K = 32

    new_elo = pd.DataFrame((score['Elo'] + K * np.sum(wins - probs, axis=1)).astype('int64'), columns=['Elo'])

    # Updating the player dataframe
    # 1. Update 'Elo' score
    players.loc[new_elo.index, 'Elo'] = new_elo['Elo']

    # 2. Increment 'Nb_matches'
    players.loc[new_elo.index, 'Nb_matches'] += 1

    # 3. Update 'Last_update'
    players.loc[new_elo.index, 'Last_update'] = score['Date']

    df_elo = pd.merge(df[['Date', 'Challenger', 'Round', 'Pts']], new_elo, on='Challenger')
    df_elo.index = df.index

    return players, df_elo, game_result


In [7]:
#tournament_date = 20211006
#game = 4

#tournament = tournaments[tournaments['Date'] == tournament_date]
#df = tournament[tournament['Round'] == game]
#compute_elo(df=df, players=players, results=results)

In [8]:
dates = tournaments['Date'].sort_values().unique().tolist()

for tournament_date in dates:
    tournament = tournaments[tournaments['Date'] == tournament_date]
    
    games = tournament['Round'].sort_values().unique().tolist()

    for game in games:
        players, df_elo, game_result = compute_elo(df=tournament[tournament['Round'] == game], players=players)
        
        tournaments.loc[df_elo.index, 'Elo'] = df_elo['Elo']
        game_result['Date'] = tournament_date
        game_result['Round'] = game

        results = pd.concat([results, game_result], ignore_index=True)
    
    print('Elo score computed for tournament: ' + str(tournament_date))

players.sort_values('Elo', ascending=False)

/var/folders/rm/zd360c_n3394v5zq_gnyj9s00000gn/T/ipykernel_34895/3148072541.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, game_result], ignore_index=True)


Elo score computed for tournament: 20211006
Elo score computed for tournament: 20211103
Elo score computed for tournament: 20211201
Elo score computed for tournament: 20220207
Elo score computed for tournament: 20220307
Elo score computed for tournament: 20220404
Elo score computed for tournament: 20220502
Elo score computed for tournament: 20220607
Elo score computed for tournament: 20220704
Elo score computed for tournament: 20220808
Elo score computed for tournament: 20220905
Elo score computed for tournament: 20221003
Elo score computed for tournament: 20221107
Elo score computed for tournament: 20221205
Elo score computed for tournament: 20221214
Elo score computed for tournament: 20230109
Elo score computed for tournament: 20230206
Elo score computed for tournament: 20230306
Elo score computed for tournament: 20230405
Elo score computed for tournament: 20230508
Elo score computed for tournament: 20230605
Elo score computed for tournament: 20230705
Elo score computed for tournamen

,Elo,Nb_matches,Last_update
Challenger,,,
Sacha,1430,132,20240916
Luca,1369,175,20241007
Lev,1356,212,20241007
Sébastien,1236,45,20231213
Ilir,1211,100,20240812
Florian,1199,123,20241007
Martin,1173,49,20241007
Romain,1152,34,20240417
Cuong,1044,59,20240916


In [9]:
tournaments.to_excel('Data/MK8_Tournaments_Elo.xlsx', sheet_name='Scores', index=False)

In [10]:
results.to_excel('Data/MK8_Tournaments_wins.xlsx', sheet_name='Scores', index=False)